In [2]:
import os
import json
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())
fmp_key = os.environ['FMP_API_KEY']
# print(fmp_key)

client : OpenAI = OpenAI()

In [3]:
# Define financial statement functions
def get_income_statement(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&limit={limit}&apikey={fmp_key}"
    response = requests.get(url)
    return json.dumps(response.json())


# Map available functions
available_functions = {
    "get_income_statement": get_income_statement,
}

In [4]:

# Define the main function
def run_assistant(user_message):
    assistant = client.beta.assistants.create(
        name="Financial Bot",
        instructions="Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends. ",
        model="gpt-3.5-turbo-1106",
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "get_income_statement",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"},
                        },
                    },
                },
            },
            # same for the rest of the financial functions
        ],
    )
      # Creating a new thread
    thread = client.beta.threads.create()

    # Adding a user message to the thr

    client.beta.threads.messages.create(
          thread_id=thread.id,
          role="user",
          content=user_message
    )

    # Running the assistant on the created thread
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        # Add run steps retrieval here
        run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
        # print("Run Steps:", run_steps)

        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    output = function_to_call(**function_args)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output,
                    })

            print("Tool Outputs:", tool_outputs)  # Add this line for debugging

            # Check if tool_outputs is not empty before submitting
            if tool_outputs:
                # Submit tool outputs and update the run
                client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )
                print("Tool outputs submitted.")  # Add this line for debugging
            else:
                print("No tool outputs to submit.")
                
        elif run.status == "completed":
            # List the messages to get the response
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            for message in messages.data:
                role_label = "User" if message.role == "user" else "Assistant"
                message_content = message.content[0].text.value
                print(f"{role_label}: {message_content}\n")
            break  # Exit the loop after processing the completed run

        elif run.status == "failed":
            print("Run failed.")
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(5)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            break
        
run_assistant('Can you compare the financial health of Microsoft and Apple over the last four years, focusing on their balance sheets and key financial ratios?')

Run is in_progress. Waiting...
Tool Outputs: [{'tool_call_id': 'call_VlqeXS09z87JZMljc7W2UDgp', 'output': '[{"date": "2023-06-30", "symbol": "MSFT", "reportedCurrency": "USD", "cik": "0000789019", "fillingDate": "2023-07-27", "acceptedDate": "2023-07-27 16:01:56", "calendarYear": "2023", "period": "FY", "revenue": 211915000000, "costOfRevenue": 65863000000, "grossProfit": 146052000000, "grossProfitRatio": 0.6892008588, "researchAndDevelopmentExpenses": 27195000000, "generalAndAdministrativeExpenses": 7575000000, "sellingAndMarketingExpenses": 22759000000, "sellingGeneralAndAdministrativeExpenses": 30334000000, "otherExpenses": 788000000, "operatingExpenses": 57740000000, "costAndExpenses": 123603000000, "interestIncome": 2994000000, "interestExpense": 1968000000, "depreciationAndAmortization": 13500000000, "ebitda": 102023000000, "ebitdaratio": 0.4814335937, "operatingIncome": 88523000000, "operatingIncomeRatio": 0.4177288064, "totalOtherIncomeExpensesNet": 788000000, "incomeBeforeTax"

KeyboardInterrupt: 